<a href="https://colab.research.google.com/github/FancyFun/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/Copy_of_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [249]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [250]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [251]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [252]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [253]:
family = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']
more = family[family['SALE_PRICE'] > 100000]
filtered = more[more['SALE_PRICE'] < 2000000]

filtered.head(5)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


#Split train data

In [286]:
import numpy as np

import matplotlib.pyplot as plt


start_date = '01/01/2019'
end_date = '03/30/2019'
start_date2 = '04/01/2019'
end_date2 = '04/30/2019'


mask = (filtered['SALE_DATE'] > start_date) & (filtered['SALE_DATE'] <= end_date)
mask2 = (filtered['SALE_DATE'] > start_date2) & (filtered['SALE_DATE'] <= end_date2)

y = filtered.SALE_PRICE
X = filtered[['YEAR_BUILT']]

Xtrain, ytrain = X.loc[mask], y.loc[mask]
Xval, yval = X.loc[mask2], y.loc[mask2]


In [287]:
Xtrain.shape

(2505, 1)

In [288]:
Xval.shape

(603, 1)

In [289]:
X.shape

(3151, 1)

#Time to do the hot encode

In [290]:
from category_encoders import OneHotEncoder
from sklearn.linear_model import LinearRegression

ohe = OneHotEncoder(use_cat_names=True)
lr = LinearRegression()

In [291]:
XT = ohe.fit_transform(Xtrain)


XYT = lr.fit(XT,ytrain)


#XT.head(5)



In [292]:
lr.predict(XT)

array([629769.52872042, 640286.73339511, 623701.91063887, ...,
       592150.29661479, 627746.9893599 , 619656.83191783])

#Select K best
###Do feature selection with SelectKBest.

In [293]:
from sklearn.feature_selection import SelectKBest
skb = SelectKBest(k='all')


ohe = OneHotEncoder(use_cat_names=True)


XT_train = ohe.fit_transform(Xtrain, ytrain)

XTT_train = skb.fit_transform(XT_train, ytrain)

model = LinearRegression(normalize=True)


model.fit(XTT_train, ytrain)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [294]:
model.score(XTT_train, ytrain)

0.0014099628164145896

#Fit a ridge regression model with multiple features. Use the normalize=True parameter

In [317]:
startt_date = '01/01/2019'
endd_date = '03/30/2019'
startt_date2 = '04/01/2019'
endd_date2 = '04/30/2019'


mask3 = (filtered['SALE_DATE'] > startt_date) & (filtered['SALE_DATE'] <= endd_date)
mask4 = (filtered['SALE_DATE'] > startt_date2) & (filtered['SALE_DATE'] <= endd_date2)

y2 = filtered.SALE_PRICE
X2 = filtered[['YEAR_BUILT','GROSS_SQUARE_FEET','BLOCK']]

Xtrain2, ytrain2 = X2.loc[mask], y2.loc[mask]
Xval2, yval2 = X2.loc[mask2], y2.loc[mask2]


In [318]:
from sklearn.linear_model import Ridge

model2 = Ridge(normalize=True)

XT_train2 = ohe.fit_transform(Xtrain2, ytrain2)

model2.fit(XT_train2,ytrain2)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
      random_state=None, solver='auto', tol=0.001)

In [319]:
model2.score(XT_train2,ytrain2)

0.17004326672488312

#Get mean absolute error for the test set

In [323]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(ytrain2, [ytrain2.mean()]*len(ytrain2))

#I know it's a lot...

214739.24214134607